### 인공신경망(ANN) 모델 살펴보기
- 구성 : 입력층 + 은닉층 + 출력층
    * 입력층 : 입력값 <--- 입력 피처수, 출력값 <--- 층의 퍼셉트론 수
    * 출력층 : 입력값 <--- 이전층의 결과수, 출력값 <--- 입력 타겟 수 (이진분류 => 1개, 다중분류 = 클래스 수, 회귀 => 1개)

In [64]:
### ===> 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [53]:
torch.manual_seed(23)

In [54]:
### ===> 데이터 생성
### ===> 피처 : 점수, 공부시간
### ===> 타겟 : 합격, 불합격
featureTS = torch.FloatTensor([[40, 4], [60, 5], [80, 6], [100, 8]])
targetTS = torch.FloatTensor([[0], [1], [1], [1]])

print(f'[featureTS] {featureTS.shape}, {featureTS.ndim}D')
print(f'[targetTS] {targetTS.shape}, {targetTS.ndim}D')

[featureTS] torch.Size([4, 2]), 2D
[targetTS] torch.Size([4, 1]), 2D


In [55]:
### ===> ANN 모델
class Net(nn.Module):
    
    # 모델 구조 설정
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 4)  # 입력층
        self.sig1 = nn.Sigmoid()

        self.layer2 = nn.Linear(4, 4)  # 은닉층
        self.sig2 = nn.Sigmoid()
        
        self.layer3 = nn.Linear(4, 1)  # 출력층
        self.sig3 = nn.Sigmoid()
    
    # 순전파 함수
    def forward(self, x):
        y1 = self.layer1(x)
        y2 = self.sig1(y1)

        y3 = self.layer2(y2)
        y4 = self.sig2(y3)

        y5 = self.layer3(y4)
        y6 = self.sig3(y5)

        return y6

In [56]:
### 모델 인스턴스 생성
model1 = Net()

model1

Net(
  (layer1): Linear(in_features=2, out_features=4, bias=True)
  (sig1): Sigmoid()
  (layer2): Linear(in_features=4, out_features=4, bias=True)
  (sig2): Sigmoid()
  (layer3): Linear(in_features=4, out_features=1, bias=True)
  (sig3): Sigmoid()
)

In [57]:
for param in model1.named_parameters():
    print(param)

('layer1.weight', Parameter containing:
tensor([[-0.1015, -0.2986],
        [-0.1097, -0.2022],
        [ 0.6473, -0.5516],
        [-0.2923,  0.5947]], requires_grad=True))
('layer1.bias', Parameter containing:
tensor([ 0.1239, -0.5234,  0.2445, -0.5618], requires_grad=True))
('layer2.weight', Parameter containing:
tensor([[ 0.2876,  0.0540, -0.0347, -0.2689],
        [-0.2786, -0.1652, -0.0459, -0.2481],
        [ 0.1310, -0.3293, -0.1878, -0.3024],
        [ 0.0466, -0.4787,  0.4049,  0.3444]], requires_grad=True))
('layer2.bias', Parameter containing:
tensor([ 0.4330, -0.2050, -0.0227, -0.0213], requires_grad=True))
('layer3.weight', Parameter containing:
tensor([[-0.1560,  0.1732,  0.1593, -0.3121]], requires_grad=True))
('layer3.bias', Parameter containing:
tensor([-0.0454], requires_grad=True))


In [58]:
### 학습 진행
### - 에포크 : 데이터 처음부터 끝까지 한번에 학습
### - 배치사이즈 : 데이터를 일정한 크기로 자른 것 (일반적 32)

### 1epoch
output = model1(featureTS)
output

tensor([[0.4558],
        [0.4558],
        [0.4558],
        [0.4558]], grad_fn=<SigmoidBackward0>)

In [59]:
### 손실계산 => 2진분류 ---> 0.5 기준으로 판별
correct = sum((output > 0.5) == targetTS)
accuracy = correct/targetTS.shape[0]

print(f'accuracy => {accuracy.item()}')

accuracy => 0.25


In [60]:
### 정밀도(precision)
correct = sum((output > 0.5))
if not correct:
    print(f'정밀도 : 0.0')

정밀도 : 0.0


In [61]:
### 재현율
correct = sum((output > 0.5) == targetTS)
print(f'재현율 : {correct.item() / sum((targetTS == 1)).item()}')


재현율 : 0.3333333333333333


#### 손실함수 즉, 정답과 예측값 사이의 오차
- 회귀 : MSE, MAE, RMSE
- 분류 : binary_croos_entropy, cross_entropy

In [62]:
print(f'targetTS \n{targetTS}')
print(f'output \n{output}')

targetTS 
tensor([[0.],
        [1.],
        [1.],
        [1.]])
output 
tensor([[0.4558],
        [0.4558],
        [0.4558],
        [0.4558]], grad_fn=<SigmoidBackward0>)


In [63]:
F.binary_cross_entropy(output, targetTS)

tensor(0.7414, grad_fn=<BinaryCrossEntropyBackward0>)

#### 학습 진행 <hr>
- (1) 모델로 학습 진행
- (2) 학습결과와 정답 비교 손실 계산
- (3) W, b 업데이트 ==> 옵티마이저

In [69]:
### ===> 학습에 필요한 준비
model2 = Net()

loss_fn = nn.BCELoss()

optimizer = optim.Adam(model2.parameters())

EPOCHS = 10

In [70]:
### ===> 학습진행

# 학습 모드로 모델 설정
model2.train()  # 배치 정규화, Dropout, 가중치 초기화 등의 작업을 할 수 있는 환경/메모리 준비

for ep in range(EPOCHS):
    # 학습
    output = model2(featureTS)

    # 손실 계산
    loss = loss_fn(output, targetTS)

    # 업데이트
    optimizer.zero_grad()  # 텐서의 grad 속성에 값을 초기화
    loss.backward()  # 학습률과 손실값으로 W, b값 업데이트 진행
    optimizer.step()  # 모델로부터 전달받은 W, b 텐서의 주소로 새로운 W, b 업데이트

    # 에포크에서 진행된 결과 출력
    print(f'[{ep}/{EPOCHS}] loss : {loss.item()}')

[0/10] loss : 0.6668105125427246
[1/10] loss : 0.665926992893219
[2/10] loss : 0.6650452613830566
[3/10] loss : 0.6641657948493958
[4/10] loss : 0.6632887721061707
[5/10] loss : 0.6624146699905396
[6/10] loss : 0.6615437865257263
[7/10] loss : 0.6606764793395996
[8/10] loss : 0.6598127484321594
[9/10] loss : 0.6589529514312744


In [ ]:
### ===> 테스트 : 모델의 성능평가/ 업데이트 발생하면 안됨!
### ===> 모델 동작 모드 설정, autograd 기능 정지, requires_grad 텐서 정지

# 테스트 모드로 모델 설정
model2.eval()

with torch.no_grad:
    # 테스트
    output = model2(featureTS) 

    # 학습
    output = model2(featureTS)

    # 손실 계산 : 테스트 데이터에 정답이 존재하는 경우 진행
    loss = loss_fn(output, targetTS)

    print(f'[TEST] loss : {loss.item()}')